## 데이터셋의 평균 길이 확인
### 입력: 폴더 경로

In [12]:
import os
import json

def process_jsonl_files(folder_path):
    text_lengths = []
    file_count = 0
    
    # 특정 폴더 내의 모든 파일을 순회
    for filename in os.listdir(folder_path):
        if filename.endswith('.jsonl'):
            file_count += 1
            file_path = os.path.join(folder_path, filename)
            
            with open(file_path, 'r', encoding='utf-8') as file:
                for line in file:
                    data = json.loads(line)
                    if 'text' in data:
                        text_length = len(data['text'])
                        text_lengths.append(text_length)
    
    if not text_lengths:
        print("No text data found in the JSONL files.")
        return
    
    # 최대 길이
    max_length = max(text_lengths)
    # 평균 길이
    average_length = sum(text_lengths) / len(text_lengths)
    # 데이터의 개수
    data_count = len(text_lengths)
    
    print(f"Number of JSONL files processed: {file_count}")
    print(f"Number of text entries: {data_count}")
    print(f"Maximum text length: {max_length}")
    print(f"Average text length: {average_length:.2f}")

# 사용할 폴더 경로를 입력하세요
folder_path = '../../data/processing_data'
process_jsonl_files(folder_path)


Number of JSONL files processed: 4
Number of text entries: 1208862
Maximum text length: 4000
Average text length: 961.99


## 데이터셋의 최대 길이 제한
### 입력: 파일 경로

In [6]:
import os
import json

def truncate_text_in_jsonl(file_path, output_path, max_length=4000):
    truncated_data = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            if 'text' in data:
                if len(data['text']) > max_length:
                    data['text'] = data['text'][:max_length]
            truncated_data.append(data)
    
    with open(output_path, 'w', encoding='utf-8') as file:
        for data in truncated_data:
            file.write(json.dumps(data, ensure_ascii=False) + '\n')

# 사용할 파일 경로와 출력 파일 경로를 입력하세요
input_file_path = '../../data/processing_data/korean-data.jsonl'
output_file_path = '../../data/processing_data/target/truncated-korean-data.jsonl'

truncate_text_in_jsonl(input_file_path, output_file_path)


## 1차 파인튜닝용 데이터셋 제작
### 구성:
1. easyread-judgment-instruct-data
2. easyword-instruct-data
3. law-terminology-instruct-data
4. truncated-korean-data

In [16]:
import os
import json
import random

def load_jsonl_files_from_folder(folder_path):
    jsonl_data = []
    
    # 폴더 내의 모든 파일 리스트를 얻기
    files = os.listdir(folder_path)
    
    # jsonl 파일만 선택
    jsonl_files = [f for f in files if f.endswith('.jsonl')]
    
    # 각 jsonl 파일 읽기
    for file_name in jsonl_files:
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                jsonl_data.append(json.loads(line))
    
    return jsonl_data

def save_jsonl_file(data, output_path):
    with open(output_path, 'w', encoding='utf-8') as file:
        for item in data:
            file.write(json.dumps(item, ensure_ascii=False) + '\n')

def main(folder_path, output_file):
    # 폴더 내의 jsonl 파일들 읽기
    jsonl_data = load_jsonl_files_from_folder(folder_path)
    
    # 데이터 랜덤으로 섞기
    random.shuffle(jsonl_data)
    
    # 결과 저장
    save_jsonl_file(jsonl_data, output_file)

if __name__ == "__main__":
    folder_path = '../../data/processing_data/target'  # jsonl 파일들이 있는 폴더 경로
    output_file = '../../data/eeve_finetuning_data/primary_dataset_nk_ngpt.jsonl'  # 결과를 저장할 파일 경로
    main(folder_path, output_file)


## 2차 파인튜닝용 데이터셋 제작
### 구성:
1. easyread-judgment-instruct-data: 90%
3. law-terminology-insturct-data: 10%

In [4]:
import os
import json
import random

def load_limited_data_from_jsonl(file_path, limit):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for i, line in enumerate(file):
            if i >= limit:
                break
            data.append(json.loads(line))
    return data

def load_limited_jsonl_files_from_folder(folder_path, file_limits):
    jsonl_data = []
    
    # 폴더 내의 모든 파일 리스트를 얻기
    files = os.listdir(folder_path)
    
    # jsonl 파일만 선택
    jsonl_files = [f for f in files if f.endswith('.jsonl')]
    
    # 각 jsonl 파일에서 제한된 개수의 데이터 읽기
    for file_name in jsonl_files:
        if file_name in file_limits:
            file_path = os.path.join(folder_path, file_name)
            limit = file_limits[file_name]
            jsonl_data.extend(load_limited_data_from_jsonl(file_path, limit))
    
    return jsonl_data

def save_jsonl_file(data, output_path):
    with open(output_path, 'w', encoding='utf-8') as file:
        for item in data:
            file.write(json.dumps(item, ensure_ascii=False) + '\n')

def main(folder_path, output_file, file_limits):
    # 폴더 내의 jsonl 파일들에서 제한된 개수만큼 데이터 읽기
    jsonl_data = load_limited_jsonl_files_from_folder(folder_path, file_limits)
    
    # 전체 데이터 개수 출력
    print(f"총 데이터 개수: {len(jsonl_data)}")
    
    # 데이터 랜덤으로 섞기
    random.shuffle(jsonl_data)
    
    # 결과 저장
    save_jsonl_file(jsonl_data, output_file)

if __name__ == "__main__":
    folder_path = '../../data/processing_data/target'  # jsonl 파일들이 있는 폴더 경로
    output_file = '../../data/finetuning_data/secondary_dataset.jsonl'  # 결과를 저장할 파일 경로
    file_limits = {
        'easyread-judgment-instruct-data.jsonl': 216000,   # file1.jsonl에서 50개의 데이터
        'not-shuffle-law-terminology-instruct-data.jsonl': 24000,  # file2.jsonl에서 100개의 데이터
        # 파일 이름과 해당 파일에서 가져올 데이터 개수를 추가
    }
    main(folder_path, output_file, file_limits)

총 데이터 개수: 240000


## 데이터셋 허깅페이스 업로드

In [7]:
from dotenv import load_dotenv
import os

load_dotenv()
token = os.getenv("HUGGINGFACE_WRITE_TOKEN")

In [17]:
from datasets import load_dataset, concatenate_datasets

dataset1 = load_dataset("json", data_files="../../data/eeve_finetuning_data/primary_dataset_nk_ngpt.jsonl")
# dataset2 = load_dataset("json", data_files="../../data/eeve_finetuning_data/secondary_dataset.jsonl")

dataset1.push_to_hub("judgment-transducer-primary-dataset-nk-ngpt")
# dataset2.push_to_hub("judgment-transducer-secondary-dataset")

Generating train split: 508379 examples [00:02, 205812.71 examples/s]
Uploading the dataset shards: 100%|██████████| 4/4 [01:29<00:00, 22.30s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/Suchae/judgment-transducer-primary-dataset-nk-ngpt/commit/d9de48b695246367dea50a5eea21f398a1a0a6fa', commit_message='Upload dataset', commit_description='', oid='d9de48b695246367dea50a5eea21f398a1a0a6fa', pr_url=None, pr_revision=None, pr_num=None)